In [1]:
!pip install Selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!apt-get install -y fonts-nanum*

'apt-get'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'apt'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'apt-get'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [4]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
from selenium.webdriver.common.keys import Keys
from collections import deque
import os

In [5]:
os.getc

'C:\\Users\\shimh\\OneDrive\\바탕 화면\\OOOA'

In [24]:
from selenium import webdriver
driver = webdriver.Chrome(r'C:\chromedriver')
driver.get('https://www.naver.com')

In [ ]:
def crawling(category):
    
    url = 'https://www.wadiz.kr/web/wsub/openfunding/reward?utm_source=rwm_gdn&utm_medium=search&utm_campaign=PC_%EB%A9%94%EC%9D%B4%EC%BB%A4&utm_content=PC_00.%EC%99%80%EB%94%94%EC%A6%88&utm_term=WADIZ&gclid=Cj0KCQjwyZmEBhCpARIsALIzmnJ4L-U5ZdU8R_6y2WpcNpXM6WVwglke6OpUP-fekXUudZSBTXWrlRAaAg0xEALw_wcB'
    webdriver.get(url)
    
    categories_sel = webdriver.find_elements_by_css_selector('#contents > section.reward-category > div > ul > div > div > li.category-slide.slick-slide.slick-current.slick-active > a')
    page_source = webdriver.page_source
    #카테고리명 가져오기
    soup = bs(webdriver.page_source, 'html.parser')
    cate = soup.findAll('span', class_ = 'desc-category')
    categories = [cat.text for cat in cate]
    
    
    search = webdriver.find_element_by_css_selector(f'#contents > section.reward-category > div > ul > div > div > li:nth-child({categories.index(category)+1}) > a')
    search.click()

    #스크롤 다운
    last_height = webdriver.execute_script("return document.body.scrollHeight")
    body = webdriver.find_element_by_tag_name("body")
    while True:
        last_height = webdriver.execute_script("document.body.scrollHeight")
        scroll_down = 0
        while scroll_down < 10:

            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.3)
            webdriver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")

            time.sleep(0.3)

            scroll_down += 1

        new_height = webdriver.execute_script("document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height  
    
    
    #titles, categories, company_names, percentages, funding_amounts, days_left
    titles, categories, company_names, percentages, funding_amounts, days_left = [],[],[],[],[],[]
    contents = webdriver.find_elements_by_class_name('ProjectCardList_item__1owJa')
    for content in contents:
        split = content.text.split('\n')
        if len(split) > 5:
            cont = deque(split)
            cont.popleft()
            title, category, company_name, target, day_left = list(cont)
        else:
            title, category, company_name, target, day_left = content.text.split('\n')

        titles.append(title)
        categories.append(category)
        company_names.append(company_name)
        percentages.append(target.split('%')[0])
        funding_amounts.append(target.split('%')[1])
        days_left.append(day_left)

    data = {
    'title':titles,
    'categories':categories,
    "company_names":company_names,
    'percentages':percentages, 
    'funding_amounts' :funding_amounts, 
    'days_left' :days_left
    } 

    
    
    n_supporters, likes = [],[]
    durations, target_amounts = [],[]
    descriptions = []
    
    #테그 가져오기
    atag = webdriver.find_elements_by_css_selector("#main-app > div.MainWrapper_content__GZkTa > div > div.RewardProjectListApp_container__1ZYeD.RewardMainProjectList_listApp__2noRS > div.ProjectCardList_container__3Y14k > div.ProjectCardList_list__1YBa2 > div a")
    list_href = [a.get_attribute('href') for a in atag]
    
#     return dataframe, list_href

    for href in list_href:
        url = href
        webdriver.get(url)
        time.sleep(0.5)
        page_source = webdriver.page_source
        soup = bs(page_source, 'html.parser')

        #n_supporters, likes
        supporter = soup.find('p',class_ = 'total-supporter').text.split('명')[0]
        n_supporters.append(supporter)
        like = soup.find('em',class_='cnt-like').text
        likes.append(like.strip())

        #durations,target_amounts
        info = soup.find('p',style='color:#00b2b2;font-size:13px;line-height:20px;margin-bottom:8px;').text.split('\n')
        duration = info[2].split(' ')[-1]
        durations.append(duration)
        target_amount = info[1].split(' ')[-1]
        target_amounts.append(target_amount)

        #descriptions
        description = soup.select("#container > div.reward-body-wrap > div > div.wd-ui-info-wrap > div.wd-ui-sub-campaign-info-container > div > div > section > div.campaign-summary")[0]
        descriptions.append(description.text.strip())
        
 
  
    #데이터 프레임 생성
 
    df = pd.DataFrame(data)
    df['percentages'] = percentages
    df['funding_amounts'] = funding_amounts
    df['n_supporters'] = n_supporters
    df['likes'] = likes
    df['descriptions'] = descriptions
    df['durations'] = durations
    df['target_amounts'] = target_amounts

        
    return df.to_csv(category+'_df')


In [ ]:
crawling('반려동물')

In [ ]:
titles

In [ ]:
#선하님 코드 확인
url = 'https://www.wadiz.kr/web/wreward/category/292?keyword=&endYn=ALL&order=recommend'
webdriver.get(url)

#스크롤 다운
last_height = webdriver.execute_script("return document.body.scrollHeight")
body = webdriver.find_element_by_tag_name("body")
while True:
    last_height = webdriver.execute_script("document.body.scrollHeight")
    scroll_down = 0
    while scroll_down < 10:

        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.3)
        webdriver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")

        time.sleep(0.3)

        scroll_down += 1

    new_height = webdriver.execute_script("document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height  

n_supporters, likes = [],[]
durations, target_amounts = [],[]
descriptions = []

atag = webdriver.find_elements_by_css_selector("#main-app > div.MainWrapper_content__GZkTa > div > div.RewardProjectListApp_container__1ZYeD.RewardMainProjectList_listApp__2noRS > div.ProjectCardList_container__3Y14k > div.ProjectCardList_list__1YBa2 > div a")
list_href = [a.get_attribute('href') for a in atag]

for href in list_href:
    url = href
    webdriver.get(url)

    page_source = webdriver.page_source
    soup = bs(page_source, 'html.parser')

    #n_supporters, likes
    supporter = soup.find('p',class_ = 'total-supporter').text.split('명')[0]
    n_supporters.append(supporter)
    like = soup.find('em',class_='cnt-like').text
    likes.append(like.strip())

In [ ]:
def timeseries_data(category):
        
    categories_sel = webdriver.find_elements_by_css_selector('#contents > section.reward-category > div')
    page_source = webdriver.page_source
    soup = bs(webdriver.page_source, 'html.parser')
    cate = soup.findAll('span', class_ = 'desc-category')
    categories = [cat.text for cat in cate]
    
    url = 'https://www.wadiz.kr/web/wsub/openfunding/reward?utm_source=rwm_gdn&utm_medium=search&utm_campaign=PC_%EB%A9%94%EC%9D%B4%EC%BB%A4&utm_content=PC_00.%EC%99%80%EB%94%94%EC%A6%88&utm_term=WADIZ&gclid=Cj0KCQjwyZmEBhCpARIsALIzmnJ4L-U5ZdU8R_6y2WpcNpXM6WVwglke6OpUP-fekXUudZSBTXWrlRAaAg0xEALw_wcB'
    webdriver.get(url)
    search = webdriver.find_element_by_css_selector(f'#contents > section.reward-category > div > ul > div > div > li:nth-child('{categories.index(category)}') > a')
    search.click()

    #스크롤 다운
    last_height = webdriver.execute_script("return document.body.scrollHeight")
    body = webdriver.find_element_by_tag_name("body")
    while True:
        last_height = webdriver.execute_script("document.body.scrollHeight")
        scroll_down = 0
        while scroll_down < 10:

            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.3)
            webdriver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")

            time.sleep(0.3)

            scroll_down += 1

        new_height = webdriver.execute_script("document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height  
    
    
    #하나씩 클릭해서 들어가야함 
    page_source = webdriver.page_source
    soup = bs(page_source, 'html.parser')
    contents = soup.findAll('div',class_='ProjectCardList_item__1owJa')
    
    for i in range(1,len(titles)+1): #len(contents)
        btn = webdriver.find_element_by_css_selector(f'div.ProjectCardList_container__3Y14k > div > div:nth-child({i}) > div > div > a > div > span')
        time.sleep(0.5)
        btn.click()
        
        #시계열 데이터
        support_page = webdriver.find_element_by_css_selector('#container > div.reward-nav > ul > li:nth-child(6)')
        time.sleep(1)
        support_page.click()

        button = webdriver.find_element_by_css_selector('#reward-static-supports-list-app > div > div > div > div.ListMoreButton_container__1rOIU > button')
        while button:
            try:
                button.click()
                time.sleep(2)
            except:
                break

        supports = []
        page_source = webdriver.page_source
        soup = bs(page_source, 'html.parser')
        supports_info = soup.findAll('div', class_ = 'RewardSupporterItem_container__1UTDZ')

        for i in supports_info:
            supports.append(i.text.split('\xa0'))
        #     데이터 전처리 필요함

        webdriver.back()
        time.sleep(0.5)
        webdriver.back()

    
    data = supports
    df = pd.DataFrame(data)
    
    return df.to_csv(category+'_tiemseires_df')

    


In [ ]:

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--start-maximized")
# chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage') #, chrome_options=chrome_options)

webdriver = webdriver.Chrome('chromedriver', chrome_options = chrome_options)

In [3]:
url = 'https://www.wadiz.kr/web/wreward/category?keyword=&endYn=ALL&order=recommend'
webdriver.get(url)

In [4]:
btn = webdriver.find_element_by_css_selector('#main-app > div.MainWrapper_content__GZkTa > div > div.RewardProjectListApp_container__1ZYeD.RewardMainProjectList_listApp__2noRS > div.ProjectCardList_container__3Y14k > div:nth-child(2) > div > button')

In [5]:
btn.text

'더보기'

In [11]:
last_height = webdriver.execute_script("return document.body.scrollHeight")
body = webdriver.find_element_by_tag_name("body")
while True:
    btn = webdriver.find_element_by_css_selector('#main-app > div.MainWrapper_content__GZkTa > div > div.RewardProjectListApp_container__1ZYeD.RewardMainProjectList_listApp__2noRS > div.ProjectCardList_container__3Y14k > div:nth-child(2) > div > button')
    if btn:
        try:
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.5)
            webdriver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")


            time.sleep(0.5)
#     new_height = webdriver.execute_script("document.body.scrollHeight")
        except:
            print('버튼 안됨/타임오류')
            break
        
    else:
        print('버튼 없음')
        break
        

버튼 안됨/타임오류


In [12]:
try:
    atag = webdriver.find_elements_by_css_selector("#main-app > div.MainWrapper_content__GZkTa > div > div.RewardProjectListApp_container__1ZYeD.RewardMainProjectList_listApp__2noRS > div.ProjectCardList_container__3Y14k > div.ProjectCardList_list__1YBa2 > div a")
    #list_href = [a.get_attribute('href') for a in atag]
except:
    print('태그 업서요')

In [13]:
list_href = [a.get_attribute('href') for a in atag]

In [14]:
len(atag)

6144

In [15]:
data = {'link' : list_href}

In [16]:
df = pd.DataFrame(data)
df

,link
0,https://www.wadiz.kr/web/wcomingsoon/rwd/10724...
1,https://www.wadiz.kr/web/wcomingsoon/rwd/10724...
2,https://www.wadiz.kr/web/campaign/detail/107995
3,https://www.wadiz.kr/web/campaign/detail/107995
4,https://www.wadiz.kr/web/campaign/detail/110747
...,...
6139,https://www.wadiz.kr/web/campaign/detail/57266
6140,https://www.wadiz.kr/web/campaign/detail/50939
6141,https://www.wadiz.kr/web/campaign/detail/50939
6142,https://www.wadiz.kr/web/campaign/detail/94697


In [ ]:
df.to_csv('61_tiemseires_df')